algoritmo que: 
* a gestão de filas de espera em hospitais envolve a classificação de risco e priorização dos pacientes;
* monitorar a quantidade de pessoas inscritas e o tempo de espera;
* definir a prioridade de atendimento. Cada cor tem um tempo de espera tolerável, que indica a gravidade do caso:
    * Vermelho: risco de morte, atendimento imediato;
     * Laranja: tempo de espera tolerável de até 10 minutos;
    * Amarelo: tempo de espera tolerável de até 20 minutos;
    * Verde: não há risco de morte, tempo de espera tolerável de até 60 minutos;
    * Azul: quadros crônicos, encaminhamento ao centro de saúde, tempo de espera tolerável de até 180 minutos.
* organize a agenda de pacientes, devendo considerar:
    * o número de consultas marcadas;
    * o tempo médio de duração e a previsão de contratempos.
* a capacitação dos profissionais de saúde influencia no tamanho das filas.

Em observação à lei abaixo, esta será considerada na implementação do algoritmo:
* A Lei 10.048/00 garante atendimento prioritário a idosos com 60 anos ou mais na administração pública.
* Também inclui pessoas do seguinte perfil: deficientes, gestantes, lactantes, pessoas com crianças de colo e obesos.
* Todavia, indivíduos com idade 80+ tem preferência sob todos os demais.

In [1]:
# !pip install faker
from faker import Faker
import random

import time

import pandas as pd
import datetime

### Gerando dados para teste

In [126]:
# Inicializando o Faker e configurando para nomes em português
fake = Faker("pt_BR")

# Inicializando as listas

lista_nomes = []          # string: primeiro nome
lista_dt_nascimento = []  # date: a data de nascimento
lista_prioridade = []     # bool: False para NÃO; True para SIM
lista_dt_entrada = []     # datetime, indicando o horário de cadastro
lista_especialidade = []  # string: especialidade médica (geral, neurologia, oftalmologista etc)
lista_tma = []            # int: minutos com o tempo médio de atendimento para cada especialidade
lista_class_risco = []    # string: vermelho, laranja, amarelo, verde, azul

lista_colunas = ['Nome', 'Nascimento', 'Prioridade', 'Entrada', 'Especialidade', 'TMA', 'Risco']
df = pd.DataFrame(columns=lista_colunas)

# Opções para os campos
especialidades_exemplo = ["geral", "neurologia", "oftalmologia", "cardiologia", "dermatologia", "pediatria"]
#class_risco_exemplo = ["vermelho", "laranja", "amarelo", "verde", "azul"]
class_risco_exemplo = [4, 3, 2, 1, 0]
# Função para gerar uma data de nascimento aleatória
def gerar_data_nascimento():
    start_date = datetime.date(1940, 1, 1)
    end_date = datetime.date(2005, 12, 31)
    return start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))

# Função para gerar uma data de entrada aleatória com horário
def gerar_data_entrada():
    now = datetime.datetime.now()
    delta_days = random.randint(0, 0)
    delta_seconds = random.randint(0, 86400)  # Número de segundos no dia
    return now - datetime.timedelta(days=delta_days, seconds=delta_seconds)

# Populando as listas com 100 registros
for _ in range(100):
    lista_nomes.append(fake.first_name())  # Gera um primeiro nome aleatório em português
    lista_dt_nascimento.append(gerar_data_nascimento())
    lista_prioridade.append(random.choice([True, False]))
    lista_dt_entrada.append(gerar_data_entrada())
    lista_especialidade.append(random.choice(especialidades_exemplo))
    lista_tma.append(random.choice([15, 30, 45, 60]))  # Tempo médio de atendimento entre 10 e 60 minutos
    lista_class_risco.append(random.choice(class_risco_exemplo))
    
lista_dados = [lista_nomes, lista_dt_nascimento, lista_prioridade, lista_dt_entrada,
               lista_especialidade, lista_tma, lista_class_risco]
    
for i, coluna in enumerate(lista_colunas):
    df[coluna] = lista_dados[i]

df

,Nome,Nascimento,Prioridade,Entrada,Especialidade,TMA,Risco
0,Bella,1965-12-09,False,2024-10-29 09:18:28.703138,neurologia,30,1
1,Renan,1960-12-17,True,2024-10-29 04:23:48.703138,neurologia,45,2
2,Luana,1948-04-13,True,2024-10-28 21:28:02.703138,cardiologia,60,3
3,Rafaela,1953-05-23,False,2024-10-29 10:48:59.703138,oftalmologia,45,4
4,Lívia,1992-02-23,True,2024-10-29 00:27:41.703138,neurologia,45,2
...,...,...,...,...,...,...,...
95,Henry Gabriel,1948-12-24,False,2024-10-29 08:11:25.706137,dermatologia,15,4
96,Marcos Vinicius,1992-10-07,False,2024-10-28 17:54:39.706137,dermatologia,45,3
97,Stella,1964-09-16,False,2024-10-28 18:51:14.706137,neurologia,15,4
98,Dante,1999-08-25,True,2024-10-29 02:42:14.706137,neurologia,60,2


### Iniciando funções e classes

In [78]:
class Fila():
    def __init__(self):
        self.data = []

    def inserir(self, x, indice=''):
        if indice == '': self.data.append(x)
        else: self.data.insert(indice, x)

    def remover(self):
        if len(self.data) > 0:
            return self.data.pop(0)

    def topo(self):
        if len(self.data) > 0:
            return self.data[0]

    def vazio(self):
        return not len(self.data) > 0
    
    def quantidade(self):
        return len(self.data)

    def exibir(self):
        return list(self.data)
    
    
    
def calc_idade(nascido):
    """
    Calcula a idade de um cliente com base na data de nascimento fornecida.

    A função calcula a idade atual do cliente subtraindo o ano de nascimento do
    ano atual. Ajusta a idade caso o mês e o dia de nascimento ainda não tenham
    ocorrido no ano atual.

    Parâmetros:
    - nascido (datetime.date): A data de nascimento do cliente.

    Retorna:
    - int: A idade do usuário em anos.
    """
    
    now = datetime.datetime.now()
    return now.year - nascido.year - ((now.month, now.day) < (nascido.month, nascido.day))

Há duas, principais, alternativas: 
1. Trabalhar com uma única fila, e dentro desta operar inserções e remanejos conforme necessário para adequar aos três graus de prioridades.
2. Fazer uso de três filas, uma para cada prioridade, mais intuitivo e fácil de implementar.

Neste primeiro momento, acabarei optando pela segunda alternativa.

In [14]:
# iniciando cada objeto fila

fila_normal = Fila()
fila_priori1 = Fila()
fila_priori2 = Fila()

In [15]:
# simulando a inserção de vários registros, em cada fila baseado na PREFERENCIA

for row in df.iterrows():    
    user = row[1].copy()
    
    idade = calc_idade(user['Nascimento'])
    user['Idade'] = idade
    senha = str(user['Especialidade'][:5]+'-'+str(row[0])).upper()
    if idade >= 80:        
        user['Senha'] = 'PE-'+senha    # PE > Prioridade Extra
        fila_priori2.inserir(user)
    elif idade >= 60 or user.Prioridade:
        user['Senha'] = 'P-'+senha
        fila_priori1.inserir(user)
    else: fila_normal.inserir(user)

In [16]:
fila_normal.quantidade()

35

In [17]:
fila_priori1.quantidade()

60

In [18]:
fila_priori2.quantidade()

5

In [ ]:
# iterar de trás para frente e ao encontrar um elemento cujo risco seja similar ao atual computado, inserir no índice seguinte

In [28]:
fila_priori2.exibir()

[Nome                            Maria Julia
 Nascimento                       1941-02-26
 Prioridade                             True
 Entrada          2024-10-28 23:29:37.784660
 Especialidade                         geral
 TMA                                      15
 Risco                                     4
 Idade                                    83
 Senha                            PE-GERAL-8
 Name: 8, dtype: object,
 Nome                                  Luiza
 Nascimento                       1943-01-25
 Prioridade                            False
 Entrada          2024-10-29 16:18:56.784660
 Especialidade                   cardiologia
 TMA                                      15
 Risco                                     1
 Idade                                    81
 Senha                           PE-CARDI-20
 Name: 20, dtype: object,
 Nome                             Ravi Lucca
 Nascimento                       1941-06-17
 Prioridade                            False
 Ent

In [27]:
fila_priori2.exibir()[0]['Risco'] == 0

False

In [134]:
risco_new_user in lista_teste

False

In [136]:
# simulando as inserções na fila, com ajustes baseado no GRAU DE RISCO
# aviso: em algum momento, as inserções DEVEM respeitar alguma medida de tempo,
# caso contrário riscos de nu 4 (azul), jamais serão atendidos

risco_new_user = 4

if risco_new_user < 5:
    if not fila_priori2.vazio():
        for i in range(fila_priori2.quantidade() - 1, -1, -1):
            usuario = fila_priori2.exibir()[i]
            print('\n', i)
            print(usuario.Nome, usuario.Risco)
            if risco_new_user <= usuario.Risco:
                print('É menor ou igual, inserindo em ', i+1)
                #fila_priori2.inserir(x, i+1)
                break

            elif i != 0 and risco_new_user > usuario.Risco:
                print('Passando!!!')
                pass
            elif i == 0:
                print('Inserir no início', i)
                fila_priori2.inserir(x)
    else: fila_priori2.inserir(x)
else: print('Erro')


 4
Théo 1
Passando!!!

 3
Dante 4
É menor ou igual, inserindo em  4


In [124]:
fila_priori2.vazio()

False

In [93]:
# lista_teste = [0, 1, 1, 2, 3, 4 ,1, 3, 0, 1, 1, 0, 3, 4]

# lista_teste.insert(len(lista_teste) , 8)

tamanho = len(lista_teste)
lista_teste.count(0)

3

In [106]:
lista_teste

[0, 1, 17, 7, 1, 2, 3, 4, 1, 3, 0, 1, 1, 0, 3, 4, 9, 9, 0, 8, 0]

In [105]:
lista_teste.insert(20, 0)

#### A se melhorar
* evitar dados duplicados em instâncias de objeto Fila()
* melhor geração de senha para guichê

fontes:
    https://www.camara.leg.br/noticias/630767-PESSOAS-MAIS-IDOSAS-PODERAO-TER-PRIORIDADE-NO-ATENDIMENTO